In [9]:
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from bs4 import BeautifulSoup
import webbrowser
import urllib.request
from lxml import html
import zipfile
import re

In [12]:
# Ensure the file exists
if not os.path.exists( "dictionary.csv" ):
    print("Missing dataset file")

In [53]:
df = pd.read_csv( "dictionary.csv")
df

,Country,Code,Population,GDP per Capita
0,Afghanistan,AFG,32526562.0,594.323081
1,Albania,ALB,2889167.0,3945.217582
2,Algeria,ALG,39666519.0,4206.031232
3,American Samoa*,ASA,55538.0,NaN
4,Andorra,AND,70473.0,NaN
5,Angola,ANG,25021974.0,4101.472152
6,Antigua and Barbuda,ANT,91818.0,13714.731960
7,Argentina,ARG,43416755.0,13431.878340
8,Armenia,ARM,3017712.0,3489.127690
9,Aruba*,ARU,103889.0,NaN


In [ ]:
df['Country']

In [18]:
def get_links(country):
    url = "https://data.worldbank.org/country/{}?year_low_desc=true".format(country)
    try:
        with urllib.request.urlopen(url) as response:
            page = response.read()
        soup = BeautifulSoup(page, 'html.parser')
        links = [ link for link in soup.find("div", {"class": "btn-item download"}).findAll('a')]
        download = re.findall('"([^"]*)"', str(links[0]))[1]
        print(download)
        return download
    except:
        print ("ERROR - "+url)


In [49]:
def download_file(country, url):
    code = url[39:42]
    country = country.replace(" ","_").replace("*","").replace(",","").replace(".","")
    if not os.path.exists( "Zips/{}.zip".format(code) ):
        urllib.request.urlretrieve(url, 'Zips/{}.zip'.format(code))  
        try:
            zip_ref = zipfile.ZipFile('Zips/{}.zip'.format(code), 'r')
            zip_ref.extractall("Unzipped")
            zip_ref.close()
            fileName = "Unzipped/API_{}_DS2_en_csv_v2.csv".format(code)
            os.rename(fileName, "Test/"+country+'.csv')
            print("Created - Test/"+country+'.csv')
        except Exception as e:
            print (country+" - ERROR")
    else:
        print("Error - File:"+"Zips/{}.zip".format(code)+" already exists.")
        

In [21]:
download_links = dict()
for country in df['Country']:
    country = country.replace("*","")
    if  ' ' in country:
        country = country.replace(' ','-')
    download_links[country] = get_links(country)

http://api.worldbank.org/v2/en/country/AFG?downloadformat=csv
http://api.worldbank.org/v2/en/country/ALB?downloadformat=csv
http://api.worldbank.org/v2/en/country/DZA?downloadformat=csv
http://api.worldbank.org/v2/en/country/ASM?downloadformat=csv
http://api.worldbank.org/v2/en/country/AND?downloadformat=csv
http://api.worldbank.org/v2/en/country/AGO?downloadformat=csv
http://api.worldbank.org/v2/en/country/ATG?downloadformat=csv
http://api.worldbank.org/v2/en/country/ARG?downloadformat=csv
http://api.worldbank.org/v2/en/country/ARM?downloadformat=csv
http://api.worldbank.org/v2/en/country/ABW?downloadformat=csv
http://api.worldbank.org/v2/en/country/AUS?downloadformat=csv
http://api.worldbank.org/v2/en/country/AUT?downloadformat=csv
http://api.worldbank.org/v2/en/country/AZE?downloadformat=csv
http://api.worldbank.org/v2/en/country/BHS?downloadformat=csv
http://api.worldbank.org/v2/en/country/BHR?downloadformat=csv
http://api.worldbank.org/v2/en/country/BGD?downloadformat=csv
http://a

http://api.worldbank.org/v2/en/country/OMN?downloadformat=csv
http://api.worldbank.org/v2/en/country/PAK?downloadformat=csv
http://api.worldbank.org/v2/en/country/PLW?downloadformat=csv
http://api.worldbank.org/v2/en/country/PAN?downloadformat=csv
http://api.worldbank.org/v2/en/country/PNG?downloadformat=csv
http://api.worldbank.org/v2/en/country/PRY?downloadformat=csv
http://api.worldbank.org/v2/en/country/PER?downloadformat=csv
http://api.worldbank.org/v2/en/country/PHL?downloadformat=csv
http://api.worldbank.org/v2/en/country/POL?downloadformat=csv
http://api.worldbank.org/v2/en/country/PRT?downloadformat=csv
http://api.worldbank.org/v2/en/country/PRI?downloadformat=csv
http://api.worldbank.org/v2/en/country/QAT?downloadformat=csv
http://api.worldbank.org/v2/en/country/ROU?downloadformat=csv
http://api.worldbank.org/v2/en/country/RUS?downloadformat=csv
http://api.worldbank.org/v2/en/country/RWA?downloadformat=csv
http://api.worldbank.org/v2/en/country/KNA?downloadformat=csv
http://a

In [33]:
df["Links"] = pd.Series(l for l in download_links.values())
df

,Country,Code,Population,GDP per Capita,Links
0,Afghanistan,AFG,32526562.0,594.323081,http://api.worldbank.org/v2/en/country/AFG?dow...
1,Albania,ALB,2889167.0,3945.217582,http://api.worldbank.org/v2/en/country/ALB?dow...
2,Algeria,ALG,39666519.0,4206.031232,http://api.worldbank.org/v2/en/country/DZA?dow...
3,American Samoa*,ASA,55538.0,NaN,http://api.worldbank.org/v2/en/country/ASM?dow...
4,Andorra,AND,70473.0,NaN,http://api.worldbank.org/v2/en/country/AND?dow...
5,Angola,ANG,25021974.0,4101.472152,http://api.worldbank.org/v2/en/country/AGO?dow...
6,Antigua and Barbuda,ANT,91818.0,13714.731960,http://api.worldbank.org/v2/en/country/ATG?dow...
7,Argentina,ARG,43416755.0,13431.878340,http://api.worldbank.org/v2/en/country/ARG?dow...
8,Armenia,ARM,3017712.0,3489.127690,http://api.worldbank.org/v2/en/country/ARM?dow...
9,Aruba*,ARU,103889.0,NaN,http://api.worldbank.org/v2/en/country/ABW?dow...


In [50]:
df[['Country','Links','Code']].apply(lambda row: download_file(row[0],row[1]) ,axis=1)

Created - Test/Afghanistan.csv
Created - Test/Albania.csv
Created - Test/Algeria.csv
Created - Test/American_Samoa.csv
Created - Test/Andorra.csv
Created - Test/Angola.csv
Created - Test/Antigua_and_Barbuda.csv
Created - Test/Argentina.csv
Created - Test/Armenia.csv
Created - Test/Aruba.csv
Created - Test/Australia.csv
Created - Test/Austria.csv
Created - Test/Azerbaijan.csv
Created - Test/Bahamas.csv
Created - Test/Bahrain.csv
Created - Test/Bangladesh.csv
Created - Test/Barbados.csv
Created - Test/Belarus.csv
Created - Test/Belgium.csv
Created - Test/Belize.csv
Created - Test/Bermuda.csv
Created - Test/Benin.csv
Created - Test/Bhutan.csv
Created - Test/Bolivia.csv
Created - Test/Bosnia_and_Herzegovina.csv
Created - Test/Botswana.csv
Created - Test/Brazil.csv
Created - Test/British_Virgin_Islands.csv
Created - Test/Brunei_Darussalam.csv
Created - Test/Bulgaria.csv
Created - Test/Burkina_Faso.csv
Created - Test/Burundi.csv
Created - Test/Cambodia.csv
Created - Test/Cameroon.csv
Created

0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
       ... 
167    None
168    None
169    None
170    None
171    None
172    None
173    None
174    None
175    None
176    None
177    None
178    None
179    None
180    None
181    None
182    None
183    None
184    None
185    None
186    None
187    None
188    None
189    None
190    None
191    None
192    None
193    None
194    None
195    None
196    None
Length: 197, dtype: object